In [1]:
import subprocess
import sys

import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''



#tf.print("PYTHON VERSION: ",sys.version)
# Install pyDOE using pip
subprocess.call(['pip', 'install', 'pyDOE'])

#!pip install pyDOE

import datetime, os

#0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import seaborn as sns 
import codecs, json
import math
#from numba import jit
# generates same random numbers each time
np.random.seed(1234)
#tf.random.set_seed(1234)
import random
import datetime
import shutil
import json
import glob
import re 

#print("TensorFlow version: {}".format(tf.__version__))
import random
import scipy.io as sio
from importlib import reload
import PINN  # python files (classes)
import pre_post
from pre_post import *
from PINN import *

Defaulting to user installation because normal site-packages is not writeable


2024-01-18 10:51:03.052304: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 10:51:05.056873: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
#@jit(nopython=True)
def generate_circles(mean_r, num_circles, std, Nx, Ny, Nz):
    # Initialize the arrays for the radii and centers of the circles
    R0 = np.zeros(num_circles)
    X_center = np.zeros(num_circles)
    Y_center = np.zeros(num_circles)
    Z_center = np.zeros(num_circles)

    # Generate the first circle randomly
    R0[0] = np.random.normal(loc=mean_r, scale=std)
    X_center[0] = np.random.randint(R0[0], Nx-R0[0])
    Y_center[0] = np.random.randint(R0[0], Ny-R0[0])
    Z_center[0] = np.random.randint(R0[0], Nz-R0[0])

    # Loop through the remaining circles and generate them one at a time
    for i in range(1, num_circles):
        # Flag to indicate whether the new circle overlaps with any existing circles
        overlaps = True
        while overlaps:
            # Generate the radius and center of the new circle randomly
            R0[i] = np.random.normal(loc=mean_r, scale=std)
            X_center[i] = np.random.randint(R0[i], Nx-R0[i])
            Y_center[i] = np.random.randint(R0[i], Ny-R0[i])
            Z_center[i] = np.random.randint(R0[i], Nz-R0[i])

            # Check the new circle against the existing circles
            overlaps = False
            for j in range(i):
                if np.sqrt((X_center[i]-X_center[j])**2 + (Y_center[i]-Y_center[j])**2 ) < (R0[i]+R0[j]): #+ (Z_center[i]-Z_center[j])**2
                    overlaps = True
                    break
    
    return R0, X_center, Y_center, Z_center

##################################################
##################################################
##################################################
if __name__ == '__main__':     ###################
##################################################
##################################################
##################################################    
    #inputs = read_inputs_from_file("Input.txt")

    # Grid parameters  
    

    Nx=65
    Ny=65
    Nt=190 # 500 if los_f
    dx = 1e-6 
    dy = dx 
    # physical parameters
    dt =1e-3
    sigma=0.24
    mu=1e-8 # 1e-8 
    delta_g= 0 
    eta=7*dx

    # Define the domain bounds
    lb = np.array([0, 0,0])
    ub = np.array([Nx*dx, Ny*dy,Nt*dt]) 

    x = np.linspace(lb[0], ub[0], Nx)
    y = np.linspace(lb[1], ub[1], Ny)
    t= np.linspace(lb[2], ub[2], Nt) 

    num_phases=1
    loc_index_0 = 0
    loc_index_1 = 1
    loc_index_2 = 2
    loc_index_3 = 3
    all_phases_indexes= [loc_index_0, loc_index_1, loc_index_2, loc_index_3]
    
    N_batches=4 # base of the pyramid
    min_batch_numbers =4 # upper surface of the pyramid
    # Training batch 
    Nbr_f_pts_max_per_batch= 50 #1000
    Nbr_f_pts_min_per_batch= 40 #100
    N_ini_max_per_batch=65  #50   
    N_ini_min_per_batch=55 #4      
    fraction_ones_per_int_pts=0.
    fraction_zeros_per_int_pts=0.
    coef_increase_points_f=2 # or decrease
    coef_increase_points_ic=2 # or decrease
    

    
    num_train_intervals=Nt
    # Define  Collocations, IC and BC points and Domain bounds
    N_ini =N_ini_max_per_batch*N_batches *num_train_intervals # Total number of data points for 'phi': IC
    N_f = Nbr_f_pts_max_per_batch * N_batches *num_train_intervals    # 100000 Total number of collocation points : domain
    N_b=N_batches*Nt   # Total number of data points for boundary BC
        
    # Total number of data points for 'phi': boundary BC
    x = np.unique(np.linspace(lb[0], ub[0], Nx))
    y = np.unique(np.linspace(lb[1], ub[1], Ny))
    t = np.unique(np.linspace(lb[2], ub[2], Nt))
    f_values = [0, 0.33, 0.67, 1]

    # Generate all combinations of x, y, t, and f
    X, Y, T, F = np.meshgrid(np.linspace(lb[0], ub[0], Nx),
                            np.linspace(lb[1], ub[1], Ny),
                            np.linspace(lb[2], ub[2], Nt),
                            f_values,
                            indexing='ij')

    # Reshape the arrays to create the test matrix
    X_phi_test=np.column_stack((X.flatten(), Y.flatten(), T.flatten()))
    np.savez('X_phi_test.npz', X_phi_test=X_phi_test)
    
    #X_phi_test = np.load('X_phi_test.npz')['arr_0']
    data = np.load('X_phi_test.npz')
    X_phi_test= data['X_phi_test']

    tb = np.linspace(start=lb[2], stop=ub[2], num=N_b, endpoint=True)
    tb = np.expand_dims(tb, axis=1)
    
    # set the saving paths and erase older results
    global pathOutput
    pathOutput = os.path.join(os.getcwd(),'save_figs')
    if not os.path.isdir(pathOutput):
        os.mkdir(pathOutput)
    global pathInput
    pathInput = os.path.join(os.getcwd(),'Initialization')
    if not os.path.isdir(pathInput):
        os.mkdir(pathInput)
    # to store the weights for each time interval 
    path_weights= os.path.join(os.getcwd(),'weights')
    if not os.path.isdir(path_weights):
        os.mkdir(path_weights)

    # load PrePost class
    Pre_Post=PrePost(X=X,T=None, lb=lb, ub=ub, Nx=Nx,Ny=Ny,dx=dx,dy=dy,x=x,y=y, eta=eta,\
            phi_true=None)

    # set the save paths and erase older results
    #Pre_Post.EraseFile(path=pathOutput)
    #Pre_Post.EraseFile(path=path_weights)
    #Pre_Post.EraseFile(path=os.path.join(os.getcwd(),'test_IC'))
    #Pre_Post.EraseFile(path=pathInput) # Initialization 

   
    # Initialize phases
    # get radius and coordinates
    R0, X_center, Y_center,Z_center =\
        generate_circles(mean_r=25*dx,num_circles=1, std=0, Nx=Nx, Ny=Ny,Nz=100)
    X_center=np.array([Nx*dx/2])
    Y_center=np.array([Ny*dx/2])
    phi_0, X_ini_all=Pre_Post.init_micro_cir(R0,X_center,Y_center, Z_center,N_ini,Nx,Ny,x,y,lb,ub) 
    phi_0=phi_0.reshape(1, Nx, Ny)
    X_ini_all=X_ini_all.reshape(1, Nx, Ny)
    # Save into a dictionary
    Initilization_Data = {'phi_0': phi_0}
    np.savez('Initilization_Data_dual.npz', **Initilization_Data)
       
    # Dual
    loaded_data = np.load('Initilization_Data_dual.npz')
    all_phases = loaded_data['phi_0']
    all_interfaces = np.zeros_like(all_phases)
    all_flags_matrix=np.zeros_like(all_phases)
    all_phases_indexes=np.zeros_like(all_phases)
    phases_indexes=all_phases_indexes
    
    tf.print("phases_indexes shape:", phases_indexes.shape)
    tf.print("all_flags_matrix shape:", all_flags_matrix.shape)
    tf.print("all_phases shape:", all_phases.shape)
    tf.print("all_interfaces shape:", all_interfaces.shape)
    
    # plot the initial micro
    Pre_Post.plot_init(all_phases,all_phases,Nx,Ny,path=pathOutput)
    
    T=eta**2 /(sigma*mu)

    # Length characteristic : eta
    # scaled arrays

    x=x/eta
    y=y/eta
    t=t/T 
    dt =dt /T 
    dx=dx/eta
    R0=R0/eta
    dy=dy/eta
    delta_g=eta*delta_g
    lb[0], ub[0]=lb[0]/eta, ub[0]/eta
    lb[1], ub[1]= lb[1]/eta, ub[1]/eta
    lb[2], ub[2]= lb[2]/T, ub[2]/T

    # get the training data
    
    X_f_train, X_ini_train_all,X_lb_train,X_ub_train,X_rtb_train,X_ltb_train,phi_ini_all =Pre_Post.set_training_data(x,y,N_ini,\
        all_phases, all_interfaces,all_flags_matrix,N_f,tb,lb,ub,path=pathOutput)
     

phases_indexes shape: (1, 65, 65)
all_flags_matrix shape: (1, 65, 65)
all_phases shape: (1, 65, 65)
all_interfaces shape: (1, 65, 65)
X_f_train: (38000, 3), X_ini_all: (4225, 3), X_lb_train: (760, 3), X_ub_train: (760, 3), X_ltb_train: (760, 3), X_rtb_train: (760, 3), phi_ini_all: (4225, 1)


In [12]:
Pre_Post.EraseFile(path=os.path.join(os.getcwd(),'save_figs'))
# load PINN class
import PINN
reload(PINN)  # mandatory to reload content at each re-call atfer modification
from PINN import *

######################################################################## 
# Build PINN 
layers = np.array([3,128,128,128,128,128,128,1])  # Network
PINN_ = Sequentialmodel(layers=layers, X_f_train=X_f_train, X_ini_train=X_ini_train_all,\
                        phases_ini_indexes=phases_indexes,all_ini_flags_matrix=all_flags_matrix,\
                        Phi_ini=all_phases,phi_ini_train=phi_ini_all, N_ini=N_ini,X_phi_test=X_phi_test,\
                        X_ini_train_all=X_ini_train_all, phi_ini_train_all=phi_ini_all,\
                            all_interfaces=all_interfaces,\
                        X_lb_train=X_lb_train, X_ub_train=X_ub_train,\
                        X_ltb_train=X_ltb_train, X_rtb_train=X_rtb_train,\
                        X=X,Y=Y,T=T,x=x,y=y,lb=lb, ub=ub, mu=mu, sigma=sigma, delta_g=delta_g,\
                        eta=eta,Nx=Nx,Ny=Ny,Nt=Nt,phi_sol=None,pinns =None,num_phases=num_phases,
                        N_batches=N_batches,\
                        min_batch_numbers = min_batch_numbers,\
                        Nbr_f_pts_max_per_batch=Nbr_f_pts_max_per_batch,\
                        Nbr_f_pts_min_per_batch=Nbr_f_pts_min_per_batch,\
                        N_ini_max_per_batch=N_ini_max_per_batch,\
                        N_ini_min_per_batch=N_ini_min_per_batch)
path_weights_all_pinns= os.path.join(os.getcwd(),'weights_all_workers_pinns')
if not os.path.isdir(path_weights_all_pinns):
    os.mkdir(path_weights_all_pinns)
pinns=PINN_.Initialize_pinns(path_weights_all_pinns)  


 ! Initilization of all workers pinns 



In [13]:
path_get_weights_all_pinns= os.path.join(os.getcwd(),'weights_all_workers_pinns')
Pre_Post.EraseFile(path="save_figs")
num_boxes = 6
time_subdomains = np.linspace(lb[2], ub[2], num_train_intervals + 1)

plot_percentages = np.around(np.linspace(0, num_train_intervals - 1, num_boxes)).astype(int)
plot_times = [0,2.8, 5.9]
time_range = np.linspace(time_subdomains[0], time_subdomains[-1], num_train_intervals)

box_size = num_train_intervals // num_boxes
num_rows = (num_boxes + 1) // 2

fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(9, 4 * num_rows), constrained_layout=True)
iterator=0
phi_evolution=[]

for idx in range(num_train_intervals ):
    if idx < num_train_intervals:
        PINN_.t_min, PINN_.t_max = time_subdomains[idx], time_subdomains[idx + 1]
    else:
        continue 
    #print(PINN_.t_min, PINN_.t_max)
    PINN_.set_weights_all_pinns(N_batches, path_get_weights_all_pinns)
    X, phi, flag_plot = PINN_.plot_evolution(N_batches, "save_figs", plot_times)
    phi_evolution.append(phi)
    if flag_plot is not None:
        if flag_plot == True:
            #print(PINN_.t_min, PINN_.t_max)
            #tf.print(X.shape, phi.shape)
            ax=axes.flat[iterator]
            iterator+=1
            scatter=ax.scatter(X[:, 0], X[:, 1], cmap=plt.get_cmap('jet'), c=phi,vmin=0,vmax=1)
            if idx==0:
                cbar = fig.colorbar(scatter, ax=ax, shrink=0.5)
                cbar.ax.set_ylabel(r'$\phi$')
            ax.set_xlabel('x')
            ax.set_ylabel('y')
            #ax.set_xlim([0,0.8])
            percentage=PINN_.t_min/time_subdomains[-1] *100
            ax.set_title(f'φ at Time: {percentage:.0f}%')

ax=axes.flat[-1]
phi_evolution=np.asarray(phi_evolution)
out_radius_vs_t=[]
for t in range(len(phi_evolution)):   
    phi=phi_evolution[t]
    phi=np.asarray(phi)
    #if t>0.5:
    phi[np.where(phi.flatten() < 1e-3)]=0
    area=len(phi[np.where(phi.flatten() > 1e-2)[0]])
    raduis=np.sqrt(np.asarray(area) /(Nx*Ny)/np.pi)
    out_radius_vs_t.append(raduis)
out_time=np.linspace(lb[2], ub[2],len(out_radius_vs_t))
ax.plot( out_time,out_radius_vs_t, "b",label=r"$PINN$",linestyle='--') #out_time,
#plt.plot( out_time[::5],R[::5], "r",label=r"$Theory$",marker='o')
ax.set_xlim([lb[2],ub[2]])
ax.set_ylabel([lb[2],ub[2]])
#ax.set_xlim([0,1])
ax.set_ylabel('Radius')
ax.set_title('Grain radius vs. Time') 
plt.legend()            
plt.savefig('save_figs/phi_evolv.jpg')


/home/selfetni/Simulations/PINN/Triple_Junction/dual_phase_V8_one_phase_4NN_no_delta_g_20_7dx_dimensionless/weights_all_workers_pinns/batch_0/pinn_0000/pinn_0000_weights_tmin_0.00000_tmax_0.04898_0.json
/home/selfetni/Simulations/PINN/Triple_Junction/dual_phase_V8_one_phase_4NN_no_delta_g_20_7dx_dimensionless/weights_all_workers_pinns/batch_1/pinn_0100/pinn_0100_weights_tmin_0.00000_tmax_0.04898_0.json
/home/selfetni/Simulations/PINN/Triple_Junction/dual_phase_V8_one_phase_4NN_no_delta_g_20_7dx_dimensionless/weights_all_workers_pinns/batch_2/pinn_0200/pinn_0200_weights_tmin_0.00000_tmax_0.04898_0.json
/home/selfetni/Simulations/PINN/Triple_Junction/dual_phase_V8_one_phase_4NN_no_delta_g_20_7dx_dimensionless/weights_all_workers_pinns/batch_3/pinn_0300/pinn_0300_weights_tmin_0.00000_tmax_0.04898_0.json
/home/selfetni/Simulations/PINN/Triple_Junction/dual_phase_V8_one_phase_4NN_no_delta_g_20_7dx_dimensionless/weights_all_workers_pinns/batch_0/pinn_0000/pinn_0000_weights_tmin_0.04898_tmax_

In [14]:
plt.figure()
out_radius_vs_t=[]
phi_evolution=np.asarray(phi_evolution)
print(phi_evolution.shape)
for t in range(len(phi_evolution)):   
    phi=phi_evolution[t]
    phi=np.asarray(phi)
    phi[np.where(phi.flatten() < 1e-3)]=0
    area=len(phi[np.where(phi.flatten() >0.01)[0]])
    raduis= np.sqrt(np.asarray(area)/Nx**2 /np.pi) 
    out_radius_vs_t.append(raduis)
out_time=np.linspace(0, ub[2] ,len(phi_evolution))
#plt.plot(PINN64_out_time[::], PINN64_out_radius_vs_t[::], "g", label=r"$PINN - 4 NN : (3,64 x 6,1)$", linestyle='--', marker='o')
plt.plot(out_time[::2],  out_radius_vs_t[::2], "red", label=r"PINN -4 NN :(3,128 x 6,1) ; $\Delta T^* = 10^{-3}$", linestyle='--', marker='s')
#plt.plot( t_th,R_th/(Nx*dx) , "b", label=r"$Theory$",linestyle='--')
#plt.plot(PF_out_time_7dx, PF_out_radius_vs_t_7dx, "m", label=r"Phase-Field ; $\Delta T^* = 10^{-4}$", linestyle='--', marker='D')

# Customize the plot
plt.xlabel("Time (dimensionless)", fontsize=12)
plt.ylabel("Radius (dimensionless)", fontsize=12)
plt.legend()
plt.savefig("test")
plt.close()

(190, 4355, 1)


In [21]:
R_th = np.zeros(Nt)

R_th[0] = R0 


for n in range(Nt-1):
    R_th[n+1] = R_th[n] + dt  * mu* (-(sigma) / R_th[n] + delta_g)

# Plotting
t_th = np.linspace(0, Nt*dt, Nt)

plt.figure()
plt.plot(t_th,R_th /(Nx*dx) )
plt.ylabel('R')
plt.savefig("Theory")

In [6]:
# Load the data from the file
import pickle
with open('PF_7dx.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# Extract loaded data
PF_out_time_7dx = loaded_data['out_time']
PF_out_radius_vs_t_7dx = loaded_data['out_radius_vs_t']